This demo is designed to work with the standard socket workers alice and bob from the other demos. It adds two numbers together

In [1]:
import syft as sy
import torch
from syft.mpc import spdz
from syft.core.frameworks.torch import _GeneralizedPointerTensor as gpt
from syft.core.frameworks.torch import _MPCTensor as mpct

Utility fuction to print chains

In [2]:
from syft.core.frameworks.torch.utils import chain_print as pp

In [3]:
hook = sy.TorchHook(verbose=False)
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
me.is_client_worker = False

compute_nodes = [bob, alice]

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [4]:
x = torch.FloatTensor([[1,2],[3,4]])
y = torch.FloatTensor([[2,3],[4,5]])

In [5]:
x_enc = spdz.encode(x)
y_enc = spdz.encode(y)

In [6]:
x_alice, x_bob = spdz.share(x_enc)
y_alice, y_bob = spdz.share(y_enc)

In [7]:
field = 2 ** 31 - 1
(x_alice + x_bob) % field


 1  2
 3  4
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [8]:
x_alice.send(alice)
x_bob.send(bob)
y_alice.send(alice)
y_bob.send(bob)

LongTensor[_PointerTensor - id:1333468834 owner:0 loc:bob id@loc:15392137671]

### GPT

In [9]:
x_pointer_tensor_dict = {alice:x_alice, bob:x_bob}
y_pointer_tensor_dict = {alice:y_alice, bob:y_bob}

In [10]:
#x_gp = gpt(x_pointer_tensor_dict,None, gpt)
#y_gp = gpt(y_pointer_tensor_dict, None, gpt)
x_gp = gpt(x_pointer_tensor_dict).on(x)
y_gp = gpt(y_pointer_tensor_dict).on(y)

In [11]:
pp(x_gp)

FloatTensor > _GeneralizedPointerTensor


In [12]:
x_gp.child.pointer_tensor_dict

{<syft.core.workers.virtual.VirtualWorker id:alice>: LongTensor[_PointerTensor - id:3165087755 owner:0 loc:alice id@loc:83618276275],
 <syft.core.workers.virtual.VirtualWorker id:bob>: LongTensor[_PointerTensor - id:5584473808 owner:0 loc:bob id@loc:59979914559]}

In [13]:
z_gp = x_gp + y_gp

In [14]:
pp(z_gp)

FloatTensor > _GeneralizedPointerTensor


In [15]:
res = []
for worker, share in z_gp.child.pointer_tensor_dict.items():
    res.append(share.get())
(res[0] + res[1]) % field


 3  5
 7  9
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

### Towards MPCT

In [11]:
x_mpc = mpct(x_gp)
y_mpc = mpct(y_gp)

In [12]:
x_mpc

[_MPCTensor - id:5254793750 owner:0]

In [17]:
x_gp2

[Head of chain]

 1  2
 3  4
[syft.core.frameworks.torch.tensor.FloatTensor of size 2x2]

In [15]:
z_mpc = x_mpc + y_mpc

AttributeError: type object 'NoneType' has no attribute 'handle_call'